In [1]:
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [27]:
df_obama = pd.read_csv('data/obama_cleaned.csv')
df_romney = pd.read_csv('data/romney_cleaned.csv')

In [22]:
df_romney.head()

,date,time,tweets,class
0,2012-10-16 00:00:00,09:38:08-05:00,insidiousmitt romneys bain helped philip morri...,-1
1,2012-10-16 00:00:00,10:14:18-05:00,you mean like romney cheated in primary,-1
2,2012-10-16 00:00:00,09:27:16-05:00,mitt romney still doesnt believe that we have ...,-1
3,2012-10-16 00:00:00,10:11:43-05:00,romneys tax plan deserves a 2nd look because h...,-1
4,2012-10-16 00:00:00,10:13:17-05:00,hope romney debate prepped w the same people a...,1


In [4]:
df_obama.head()

,date,time,tweets,class
0,2012-10-16 00:00:00,10:28:53-05:00,kirkpatrick who wore a baseball cap embroidere...,0
1,2012-10-16 00:00:00,10:04:30-05:00,obama debates that cracker ass cracker tonight...,1
2,2012-10-16 00:00:00,09:50:08-05:00,youre missing the point im afraid you do not u...,0
3,2012-10-16 00:00:00,10:00:16-05:00,i was raised as a democrat left the party year...,-1
4,2012-10-16 00:00:00,09:48:07-05:00,the obama camp cant afford to lower expectatio...,0


In [1]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="cardiffnlp/twitter-roberta-base-sentiment-latest")

/Users/aarshpatel/Downloads/DMTM (CS 583)/Project 2/.venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/aarshpatel/Downloads/DMTM (CS 583)/Project 2/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSe

In [17]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
#model.save_pretrained(MODEL)

sc = {'positive': [], 'neutral': [], 'negative': []}

for tweet in df_obama['tweets']:
    encoded_input = tokenizer(tweet, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    ranking = np.argsort(scores)
    ranking = ranking[::-1]

    for i in range(scores.shape[0]):
        l = config.id2label[ranking[i]]
        s = scores[ranking[i]]
        sc[l].append(s)
        # print(f"{l} {np.round(float(s), 4)}")


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [27]:
df_obama['positive'] = sc['positive']
df_obama['neutral'] = sc['neutral']
df_obama['negative'] = sc['negative']
df_obama.rename(columns={'positive': 'pos', 'neutral': 'neu', 'negative': 'neg'}, inplace=True)

In [36]:
preds = list()
for idx, row in df_obama.iterrows():
    if row['pos'] >= row['neu'] and row['pos'] >= row['neg']:
        preds.append(1)
    elif row['neg'] >= row['neu'] and row['neg'] > row['pos']:
        preds.append(-1)
    elif row['neu'] > row['pos'] and row['neu'] > row['neg']:
        preds.append(0)
df_obama['pred'] = preds

In [40]:
acc = accuracy_score(df_obama['class'], df_obama['pred'])
prec = precision_score(df_obama['class'], df_obama['pred'], average = None, zero_division = np.nan)
rec = recall_score(df_obama['class'], df_obama['pred'], average = None)
f1 = f1_score(df_obama['class'], df_obama['pred'], average = None)
print("Accuracy:", acc)
print("Precision:", prec)
print("Recall:", rec)
print("F1:", f1)

Accuracy: 0.4916429587482219
Precision: [0.60100063 0.42036986 0.69900498]
Recall: [0.48831301 0.77035913 0.16736152]
F1: [0.53882815 0.54392857 0.27006247]
